In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import random
import math
import torch
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

In [4]:
device = torch.device('cpu')
output_folder = 'NL'
# 读取数据集为训练
def read_data(path):
    data = pd.read_csv(path)
    x = data.values[:, :-1]  # 特征
    y = data.values[:, -1]   # 目标
    return x, y

# 准备数据
x_train, y_train = read_data(f'data_exp/{output_folder}/train_set.csv')
x_test, y_test = read_data(f'data_exp/{output_folder}/test_set.csv')
x_val, y_val = read_data(f'data_exp/{output_folder}/val_set.csv')

# 转换为 PyTorch 张量
def d2tensor(x):
    return torch.FloatTensor(x).to(device)

train_x = d2tensor(x_train)
train_y = d2tensor(y_train).reshape(-1, 1)
val_x = d2tensor(x_val)
val_y = d2tensor(y_val).reshape(-1, 1)
test_x = d2tensor(x_test)
test_y = d2tensor(y_test).reshape(-1, 1)

# 输出确认
print("训练集形状:", train_x.shape, train_y.shape)
print("验证集形状:", val_x.shape, val_y.shape)
print("测试集形状:", test_x.shape, test_y.shape)

训练集形状: torch.Size([28098, 13]) torch.Size([28098, 1])
验证集形状: torch.Size([3513, 13]) torch.Size([3513, 1])
测试集形状: torch.Size([3512, 13]) torch.Size([3512, 1])
